In [1]:
import os
import tarfile
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPool2D, BatchNormalization, Flatten, Dropout
import os
from flask import Flask, render_template, request, redirect, url_for
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from werkzeug.utils import secure_filename

In [ ]:
DATA_PATH = r'/content/drive/MyDrive/cifar10.tgz'
FILE_PATH = r'/content/data/cifar10'
TRAIN_PATH = os.path.join(FILE_PATH, 'train')
VAL_PATH = os.path.join(FILE_PATH, 'test')
TARGET_SIZE = (32,32)
BATCH_SIZE = 64
EPOCHS = 10



In [ ]:
with tarfile.open(DATA_PATH, 'r:gz') as tar:
  tar.extractall(path = './data')



In [ ]:
classes = os.listdir(TRAIN_PATH)
print(len(classes))



In [ ]:
image_path = os.path.join(VAL_PATH,'airplane','0001.png')
image_metrix = cv2.imread(image_path)



In [ ]:


plt.figure(figsize=(5,5))
plt.imshow(image_metrix)



In [ ]:
train_datagen_object = ImageDataGenerator(rescale = 1./32,
                                          width_shift_range = 0.7,
                                          height_shift_range = 0.7,
                                          shear_range = 0.7,
                                          zoom_range = 0.5,
                                          horizontal_flip = True,
                                          vertical_flip = True)
val_datagen_object = ImageDataGenerator(rescale = 1./32)



In [ ]:
train_datagen = train_datagen_object.flow_from_directory(directory = TRAIN_PATH,
                                                         target_size = TARGET_SIZE,
                                                         batch_size = BATCH_SIZE)
val_datagen = val_datagen_object.flow_from_directory(directory = VAL_PATH,
                                                     target_size = TARGET_SIZE,
                                                     batch_size = BATCH_SIZE)



In [ ]:

model = Sequential()
model.add(Input(TARGET_SIZE.__add__((3,)),
                name = 'InputLayer'))
model.add(Conv2D(filters = 32,
                 kernel_size = (3,3),
                 padding = 'same',
                 activation = 'relu',
                 kernel_initializer = 'he_normal',
                 name = 'ConvLayer1'))
model.add(MaxPool2D(pool_size=(2,2),
                    name = 'PoolingLayer1'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 64,
                 kernel_size = (3,3),
                 padding = 'same',
                 activation = 'relu',
                 kernel_initializer = 'he_normal',
                 name = 'ConvLayer2'))
model.add(MaxPool2D(pool_size=(2,2),
                    name = 'PooingLayer2'))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(units = 512,
                activation = 'relu',
                kernel_initializer = 'he_normal',
                name = 'HiddenLayer1'))
model.add(Dropout(rate = 0.3))
model.add(BatchNormalization())
model.add(Dense(units = 1024,
                activation = 'relu',
                kernel_initializer = 'he_normal',
                name = 'HiddenLayer2'))
model.add(Dropout(rate = 0.3))
model.add(BatchNormalization())
model.add(Dense(units = 10,
                activation = 'softmax',
                kernel_initializer = 'he_normal',
                name = 'OutputLayer'))



In [ ]:
model.compile(optimizer = 'Adam',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy',
                         'Precision',
                         'Recall'])



In [ ]:
history_object = model.fit(train_datagen,
                          epochs = EPOCHS,
                          batch_size = BATCH_SIZE,
                          validation_data = val_datagen,
                          workers = 10,
                          use_multiprocessing = True)

In [ ]:
model.save('my_cnn_model.tf', save_format = 'tf')

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Image Classifier</title>
</head>
<body>
    <h1>Image Classifier</h1>
    <form action="/upload" method="post" enctype="multipart/form-data">
        <input type="file" name="file" accept=".png, .jpg, .jpeg" required>
        <input type="submit" value="Upload and Predict">
    </form>
</body>
</html>


In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Result</title>
</head>
<body>
    <h2>Result:</h2>
    <p>Predicted Category: {{ prediction }}</p>
    <a href="/">Go Back</a>
</body>
</html>
